<a href="https://colab.research.google.com/github/CD-AC/AIEngennier-Jurid_IA/blob/main/Jurid_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jurid-IA

un asistente de inteligencia artificial experto diseñado para actuar como un único punto de acceso inteligente a todo el conocimiento de la organización. La solución se fundamenta en una arquitectura moderna de RAG (Retrieval-Augmented Generation), orquestada con el framework LangChain. Para automatizar la ingesta de datos, se implementó un pipeline ETL robusto con Apache Airflow, que extrae, transforma y procesa documentos de las distintas fuentes.

## Instalar Dependencias

In [1]:
!pip install -q langchain langchain_openai langchain_community gradio openai python-dotenv beautifulsoup4 pinecone-client langchain-pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.1/221.1 

## Importar Dependencias y Configuración Inicial

In [2]:
import os
import glob
import time
import gradio as gr
from pinecone import Pinecone as PineconeClient, ServerlessSpec

# LangChain Imports
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_pinecone import Pinecone
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

## Configuración OpenAI - Pinecone

In [11]:
class Config:
    # OpenAI
    LLM_MODEL_NAME = "gpt-4o-mini"
    EMBEDDING_MODEL_NAME = "text-embedding-3-small"
    EMBEDDING_DIMENSION = 1536

    # Directorio
    KNOWLEDGE_BASE_DIR = "/content/knowledge-base"

    # Pinecone
    PINECONE_INDEX_NAME = "jurid-ia-idx"

    # Parámetros text spliter
    CHUNK_SIZE = 1000
    CHUNK_OVERLAP = 150

    # Parámetros de Búsqueda del Retriever
    SEARCH_K = 5

## Configuración de API_KEY

In [4]:
# Cargar API Key de OpenAI
openai_api_key_file = '/content/OPENAI_API_KEY.txt'
try:
    with open(openai_api_key_file, 'r') as f:
        os.environ['OPENAI_API_KEY'] = f.read().strip()
    print("API Key de OpenAI cargada.")
except Exception as e:
    print(f"Error al cargar la API Key de OpenAI: {e}.")
    raise

# Cargar API Key de Pinecone
pinecone_api_key_file = '/content/PINECONE_API_KEY.txt'
try:
    with open(pinecone_api_key_file, 'r') as f:
        os.environ['PINECONE_API_KEY'] = f.read().strip()
    print("API Key de Pinecone cargada.")
except Exception as e:
    print(f"Error al cargar la API Key de Pinecone: {e}.")
    raise

API Key de OpenAI cargada.
API Key de Pinecone cargada.


## Funciones Modulares del Proyecto

In [12]:
# Valida la carpeta knowledge-base
def validar_base_conocimiento():
    if not os.path.isdir(Config.KNOWLEDGE_BASE_DIR):
        error_message = (
            f"Error: El directorio '{Config.KNOWLEDGE_BASE_DIR}' no fue encontrado."
        )
        raise FileNotFoundError(error_message)
    print(f"Directorio '{Config.KNOWLEDGE_BASE_DIR}' encontrado.")

In [32]:
# Crea o actualiza una base de datos vectorial en Pinecone.

"""
1. Carga y divide los documentos.
2. Inicializa la conexión con Pinecone.
3. Comprueba si el índice ya existe. Si existe, lo elimina para empezar de cero.
4. Crea un nuevo índice con la configuración correcta.
5. Genera los embeddings y los sube al índice de Pinecone.
"""

def crear_base_conocimiento(knowledge_dir: str, index_name: str):
    print("Cargando y procesando documentos...")
    documents = []
    for item_path in glob.glob(f"{knowledge_dir}/*"):
        if os.path.isdir(item_path):
            loader = DirectoryLoader(item_path, glob="**/*.md", loader_cls=TextLoader, loader_kwargs={'encoding': 'utf-8'}, show_progress=True)
            documents.extend(loader.load())
        elif os.path.isfile(item_path) and item_path.endswith('.md'):
             loader = TextLoader(item_path, encoding='utf-8')
             documents.extend(loader.load())

    if not documents:
        raise ValueError(f"No se encontraron documentos en el directorio: {knowledge_dir}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=Config.CHUNK_SIZE, chunk_overlap=Config.CHUNK_OVERLAP)
    chunks = text_splitter.split_documents(documents)
    print(f"Total de fragmentos (chunks) creados: {len(chunks)}")

    embeddings = OpenAIEmbeddings(model=Config.EMBEDDING_MODEL_NAME)

    # --- Lógica de Pinecone ---

    print(f"Inicializando cliente de Pinecone...")
    pc = PineconeClient()

    # Comprobar si el índice ya existe
    if index_name in [index.name for index in pc.list_indexes()]:
        print(f"Índice '{index_name}' encontrado.")
        pc.delete_index(index_name)
        time.sleep(5)

    # Crear un nuevo índice
    print(f"Creando un nuevo índice en Pinecone: '{index_name}'...")
    pc.create_index(
        name=index_name,
        dimension=Config.EMBEDDING_DIMENSION,
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

    # Esperar a que el índice esté listo
    while not pc.describe_index(index_name).status['ready']:
        print("Preparando el índice.")
        time.sleep(1)

    print(f"Añadiendo {len(chunks)} fragmentos al índice de Pinecone.")

    # LangChain genera los embeddings y los carga al índice
    vectorstore = Pinecone.from_documents(
        documents=chunks,
        embedding=embeddings,
        index_name=index_name
    )

    print(f"\nBase de datos vectorial en Pinecone creada y poblada.")
    return vectorstore

In [29]:
#  Configura la cadena de conversación.
def configurar_cadena_conversacional(vectorstore):
    llm = ChatOpenAI(model=Config.LLM_MODEL_NAME, temperature=0)
    retriever = vectorstore.as_retriever(search_kwargs={"k": Config.SEARCH_K})
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory,
    )

    print("Cadena de conversación configurada.")
    return conversation_chain

In [30]:
# Crea y lanza la interfaz de chat de Gradio.
def iniciar_chat_gradio(conversation_chain):
    def chat_function_with_sources(question, history):
        try:
            # 1. Preparar el historial para LangChain
            langchain_history = []
            for human, ai in history:
                if human:
                    langchain_history.append({"role": "user", "content": human})
                if ai:
                    langchain_history.append({"role": "assistant", "content": ai})

            # 2. Invocar la cadena de conversación
            result = conversation_chain.invoke({"question": question, "chat_history": langchain_history})
            answer = result["answer"]

            # 3. Extraer y formatear las fuentes
            source_docs = result.get("source_documents", [])
            source_text = ""
            if source_docs:
                unique_sources = sorted(list(set(os.path.basename(doc.metadata.get('source', 'Desconocido')) for doc in source_docs)))
                source_list = "\\n".join([f"- `{source}`" for source in unique_sources])
                source_text = f"\\n\\n---\\n**Fuentes Consultadas:**\\n{source_list}"

            return answer, source_text

        except Exception as e:
            error_message = f"Ocurrió un error: {e}"
            return error_message, ""

    # Construcción de la Interfaz
    with gr.Blocks(theme='soft', title="Jurid-IA") as interface:
        gr.Markdown(
            """
            <div style="text-align: center;">
                <h1>⚖️ Jurid-IA Asistente Jurídico de IA </h1>
                <p>Tu experto entrenado con los documentos internos del Banco XYZ.</p>
            </div>
            """
        )

        # Área principal del chat y las fuentes
        with gr.Row():
            with gr.Column(scale=4):
                chatbot = gr.Chatbot(
                    label="Chat",
                    height=550
                )
                sources = gr.Markdown(label="Fuentes", value="*Las fuentes consultadas para la última respuesta aparecerán aquí.*")

            with gr.Column(scale=1):
                gr.Markdown("### Ejemplos de Preguntas")
                with gr.Row():
                    textbox = gr.Textbox(
                        placeholder="Escribe tu pregunta aquí...",
                        show_label=False,
                        container=False,
                        scale=7,
                    )
                    submit_btn = gr.Button("Enviar", variant="primary", scale=1)

                examples = gr.Examples(
                    examples=[
                        "¿Cuál es la política de teletrabajo?",
                        "¿Cuáles son los términos de pago del contrato con 'Tech Innovators'?",
                        "¿Cuál es el límite para aceptar regalos según el código de conducta?"
                    ],
                    inputs=[textbox],
                    label="Haz clic en un ejemplo para probar",
                    fn=lambda x: x,
                    outputs=[textbox]
                )


        # --- Lógica de Interacción ---

        # Función que se ejecuta al enviar el formulario
        def handle_submit(question, history):
            response, source_text = chat_function_with_sources(question, history)
            history.append((question, response))
            return "", history, source_text

        # Conectar el botón de envío
        submit_btn.click(
            fn=handle_submit,
            inputs=[textbox, chatbot],
            outputs=[textbox, chatbot, sources],
            queue=True
        )

        # Conectar la acción de presionar "Enter"
        textbox.submit(
            fn=handle_submit,
            inputs=[textbox, chatbot],
            outputs=[textbox, chatbot, sources],
            queue=True
        )

    # Lanzar la interfaz
    interface.launch(debug=True, share=True)

## Ejecución del Proyecto

In [33]:
# Función principal que orquesta todo el proceso.

def main():
    try:
        # 1. Valida que los documentos fuente existan
        validar_base_conocimiento()

        # 2. Crea la base de datos vectorial en Pinecone
        vectorstore = crear_base_conocimiento(
            knowledge_dir=Config.KNOWLEDGE_BASE_DIR,
            index_name=Config.PINECONE_INDEX_NAME
        )

        # 3. Configura la cadena de conversación con el vectorstore de Pinecone
        conversation_chain = configurar_cadena_conversacional(vectorstore)

        # 4. Inicia la interfaz de usuario de Gradio para el chat
        iniciar_chat_gradio(conversation_chain)

    except Exception as e:
        print(f"\\nHa ocurrido un error en la ejecución: {e}")
        import traceback
        traceback.print_exc()

# Punto de entrada para la ejecución del script
if __name__ == "__main__":
    main()

Directorio '/content/knowledge-base' encontrado.
Cargando y procesando documentos...


100%|██████████| 5/5 [00:00<00:00, 8348.54it/s]

Total de fragmentos (chunks) creados: 59
Inicializando cliente de Pinecone...


Índice 'jurid-ia-idx' encontrado.
Creando un nuevo índice en Pinecone: 'jurid-ia-idx'...
Añadiendo 59 fragmentos al índice de Pinecone.

Base de datos vectorial en Pinecone creada y poblada.
Cadena de conversación configurada.


/tmp/ipython-input-2570773561.py:46: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9b139fdca222084b47.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9b139fdca222084b47.gradio.live
